<a href="https://colab.research.google.com/github/Asmit-Dhakal/Deep_learning/blob/main/01_pytorch_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **# Pytorch Work-flow**

In [7]:
topics={
    1:"data(prepare and load)",
    2: "build model",
    3: "fitting the model to the data(training)",
    4: "making prediciton and evaluating a model",
    5: "saving and loading model",
    6: "conclusion"
}

In [8]:
topics

{1: 'data(prepare and load)',
 2: 'build model',
 3: 'fitting the model to the data(training)',
 4: 'making prediciton and evaluating a model',
 5: 'saving and loading model',
 6: 'conclusion'}

In [9]:
import torch
from torch import nn # nn contain all building block for neural network
import matplotlib.pyplot as plt

# Check pytorch version
torch.__version__



'2.2.1+cu121'

Machine learning has tow parts:
1. Get data into a numerical representation
2. Build a model to learn patterns in that numerical representation

In [4]:
# let us use linear regression formula and nake straight line

## Create *known* parameters
weight=0.7
bias=0.3

# Create
start = 0
end =1
step =0.02
X = torch.arange(start,end,step).unsqueeze(dim=1)
y=weight*X + bias
X[:10], y[:10]



(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

In [5]:
len(X) ,len(y)

(50, 50)

In [11]:

# Spliting data into traing and test set .(one of the most important concepts in machine learning in general)
